In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
datasets_path = '../data'
movies_path = '/movies'
raw_path = '/raw'
processed_path = '/processed'

movies_metadata = pd.read_csv(datasets_path + movies_path + raw_path + '/movies_metadata.csv')
movies_credits = pd.read_csv(datasets_path + movies_path + raw_path + '/credits.csv')
movies_ratings = pd.read_csv(datasets_path + movies_path + raw_path + '/ratings.csv')

C:\Users\eysto\AppData\Local\Temp\ipykernel_6512\2683035399.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv(datasets_path + movies_path + raw_path + '/movies_metadata.csv')


In [3]:
eda_movies_metadata = movies_metadata.copy()
eda_movies_metadata.drop(['belongs_to_collection', 'homepage', 'overview', 'poster_path', 'tagline', 'production_companies', 'production_countries'], axis=1, inplace=True)
eda_movies_metadata

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,release_date,revenue,runtime,spoken_languages,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,tt6209470,fa,رگ خواب,0.072051,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Subdue,False,4.0,1.0
45462,False,0,"[{'id': 18, 'name': 'Drama'}]",111109,tt2028550,tl,Siglo ng Pagluluwal,0.178241,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,Century of Birthing,False,9.0,3.0
45463,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,tt0303758,en,Betrayal,0.903007,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Betrayal,False,3.8,6.0
45464,False,0,[],227506,tt0008536,en,Satana likuyushchiy,0.003503,1917-10-21,0.0,87.0,[],Released,Satan Triumphant,False,0.0,0.0


In [4]:
def extract_json_feature(df, column_name):
    def safe_parse(x):
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except (ValueError, SyntaxError):
                return []
        return []

    parsed_data = df[column_name].apply(safe_parse)

    extracted_names = parsed_data.apply(lambda x: [item['name'] for item in x] if isinstance(x, list) else [])

    mlb = MultiLabelBinarizer()
    encoded_features = pd.DataFrame(
        mlb.fit_transform(extracted_names),
        columns=[f"{column_name}_{name}" for name in mlb.classes_],
        index=df.index
    )

    return encoded_features

In [5]:
def extract_json_feature_sparse(df, column_name):
    def safe_parse(x):
        if isinstance(x, str):
            try:
                return ast.literal_eval(x)
            except (ValueError, SyntaxError):
                return []
        return []

    parsed_data = df[column_name].apply(safe_parse)

    extracted_names = parsed_data.apply(lambda x: [item['name'] for item in x] if isinstance(x, list) else [])

    mlb = MultiLabelBinarizer(sparse_output=True)  # Enable sparse matrix
    encoded_features = mlb.fit_transform(extracted_names)

    # Convert to Pandas Sparse DataFrame to save memory
    encoded_df = pd.DataFrame.sparse.from_spmatrix(
        encoded_features,
        columns=[f"{column_name}_{name}" for name in mlb.classes_],
        index=df.index
    )

    return encoded_df


In [6]:
json_features = ['genres', 'spoken_languages']
learning_movies_metadata = eda_movies_metadata.copy()

for feature in json_features:
    learning_movies_metadata = pd.concat([learning_movies_metadata, extract_json_feature_sparse(learning_movies_metadata, feature)], axis=1)
    learning_movies_metadata.drop(feature, axis=1, inplace=True)

In [7]:
learning_movies_metadata

,adult,budget,id,imdb_id,original_language,original_title,popularity,release_date,revenue,runtime,...,spoken_languages_বাংলা,spoken_languages_ਪੰਜਾਬੀ,spoken_languages_தமிழ்,spoken_languages_తెలుగు,spoken_languages_ภาษาไทย,spoken_languages_ქართული,spoken_languages_广州话 / 廣州話,spoken_languages_日本語,spoken_languages_普通话,spoken_languages_한국어/조선말
0,False,30000000,862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,...,0,0,0,0,0,0,0,0,0,0
1,False,65000000,8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249.0,104.0,...,0,0,0,0,0,0,0,0,0,0
2,False,0,15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,...,0,0,0,0,0,0,0,0,0,0
3,False,16000000,31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,...,0,0,0,0,0,0,0,0,0,0
4,False,0,11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,439050,tt6209470,fa,رگ خواب,0.072051,NaN,0.0,90.0,...,0,0,0,0,0,0,0,0,0,0
45462,False,0,111109,tt2028550,tl,Siglo ng Pagluluwal,0.178241,2011-11-17,0.0,360.0,...,0,0,0,0,0,0,0,0,0,0
45463,False,0,67758,tt0303758,en,Betrayal,0.903007,2003-08-01,0.0,90.0,...,0,0,0,0,0,0,0,0,0,0
45464,False,0,227506,tt0008536,en,Satana likuyushchiy,0.003503,1917-10-21,0.0,87.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
learning_movies_credits = pd.concat([movies_credits['id'].astype(str), extract_json_feature_sparse(movies_credits, 'crew')], axis=1)

In [9]:
learning_movies_credits

,id,crew_\tCheung Chi-Sing,crew_\tJiang Jun,crew_\tMak Tin-Shu,crew_\tPertti Pasanen,crew_\tSo Lam,crew_ Anshuman Mahaley,crew_ Dale Trevillion\t,crew_ Davide Manuli,crew_ E.W. Swackhamer,...,crew_정성철,crew_정아연,crew_조윤진,crew_조진아,crew_조희란,crew_진모영,crew_최광식,crew_최세연,crew_한소현,crew_허종호
0,862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8844,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15602,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,31357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11862,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45471,439050,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45472,111109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45473,67758,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45474,227506,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
print(learning_movies_credits.dtypes)

id                                  object
crew_\tCheung Chi-Sing    Sparse[int32, 0]
crew_\tJiang Jun          Sparse[int32, 0]
crew_\tMak Tin-Shu        Sparse[int32, 0]
crew_\tPertti Pasanen     Sparse[int32, 0]
                                ...       
crew_진모영                  Sparse[int32, 0]
crew_최광식                  Sparse[int32, 0]
crew_최세연                  Sparse[int32, 0]
crew_한소현                  Sparse[int32, 0]
crew_허종호                  Sparse[int32, 0]
Length: 157615, dtype: object


In [10]:
common_ids = set(learning_movies_metadata['id']).intersection(learning_movies_credits['id'].astype(str))
learning_movies_metadata = learning_movies_metadata[learning_movies_metadata['id'].isin(common_ids)]
learning_movies_credits = learning_movies_credits[learning_movies_credits['id'].isin(common_ids)]

In [11]:
learning_movies_combined = pd.merge(learning_movies_metadata, learning_movies_credits, on='id', how='inner')

In [15]:
learning_movies_credits['vote_average'] = learning_movies_metadata['vote_average']

In [16]:
def write_dfs_as_csv(dfs):
    for filename, df in dfs.items():
        file_path = datasets_path + movies_path + processed_path + filename
        df.to_csv(file_path, index=False)

In [17]:
write_dfs_as_csv(dfs=
{
    '/eda_movies_metadata.csv':eda_movies_metadata,
    '/learning_movies_metadata.csv':learning_movies_metadata,
    # '/learning_movies_credits.csv':learning_movies_credits,
    # '/learning_movies_combined.csv':learning_movies_combined,
})

KeyboardInterrupt: 

In [13]:
learning_movies_combined

,adult,budget,id,imdb_id,original_language,original_title,popularity,release_date,revenue,runtime,...,crew_정성철,crew_정아연,crew_조윤진,crew_조진아,crew_조희란,crew_진모영,crew_최광식,crew_최세연,crew_한소현,crew_허종호
0,False,30000000,862,tt0114709,en,Toy Story,21.946943,1995-10-30,373554033.0,81.0,...,0,0,0,0,0,0,0,0,0,0
1,False,65000000,8844,tt0113497,en,Jumanji,17.015539,1995-12-15,262797249.0,104.0,...,0,0,0,0,0,0,0,0,0,0
2,False,0,15602,tt0113228,en,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,...,0,0,0,0,0,0,0,0,0,0
3,False,16000000,31357,tt0114885,en,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,...,0,0,0,0,0,0,0,0,0,0
4,False,0,11862,tt0113041,en,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,False,0,439050,tt6209470,fa,رگ خواب,0.072051,NaN,0.0,90.0,...,0,0,0,0,0,0,0,0,0,0
45534,False,0,111109,tt2028550,tl,Siglo ng Pagluluwal,0.178241,2011-11-17,0.0,360.0,...,0,0,0,0,0,0,0,0,0,0
45535,False,0,67758,tt0303758,en,Betrayal,0.903007,2003-08-01,0.0,90.0,...,0,0,0,0,0,0,0,0,0,0
45536,False,0,227506,tt0008536,en,Satana likuyushchiy,0.003503,1917-10-21,0.0,87.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for col in learning_movies_combined.select_dtypes(include=["number"]).columns:
    if (learning_movies_combined[col] == 0).sum() / len(learning_movies_combined[col]) > 0.8:
        learning_movies_combined[col] = pd.arrays.SparseArray(learning_movies_combined[col], fill_value=0)

In [19]:
learning_movies_credits.to_pickle(datasets_path + movies_path + processed_path + '/learning_movies_credits.pkl')

In [20]:
learning_movies_combined.to_pickle(datasets_path + movies_path + processed_path + '/learning_movies_combined.pkl')